In [264]:
import pandas as pd    
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from lxml import html
import time

In [282]:
# Set up the Selenium webdriver
driver = webdriver.Chrome()
url = 'https://www.google.com/maps/place/GetGo+Carsharing/@1.2943816,103.7871263,15z/data=!4m2!3m1!1s0x0:0x4dc7314e4bdb6399?sa=X&ved=2ahUKEwi5zrin0oyEAxWr1zgGHc19B80Q_BJ6BAhcEAA'
driver.get(url)

# Get the page source
page_source = driver.page_source

# Parse the HTML using lxml
tree = html.fromstring(page_source)

# Find the element with XPath
xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span'
reviews_element = tree.xpath(xpath)

# Extract the total number of reviews (assuming it's a single element)
if reviews_element:
    # Extract the text content and use regular expression to get the number
    total_reviews_text = reviews_element[0].text_content()
    total_reviews = int(''.join(filter(str.isdigit, total_reviews_text)))
    print(f'Total number of reviews: {total_reviews}')
else:
    print('Element not found')

# Close the Selenium webdriver
driver.quit()

Total number of reviews: 2398


In [283]:
# Launch web driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [284]:
# Load google maps url
# url = 'https://www.google.com/maps/place/CarTimes+Autolution+@+SinMing/@1.3595828,103.8361185,17z/data=!3m2!4b1!5s0x31da172e4e7d5c0b:0xcdc65f29ab85b4a5!4m6!3m5!1s0x31da17c644cef3a1:0x187163161ac34209!8m2!3d1.3595774!4d103.8386934!16s%2Fg%2F11ght0mjh_?entry=ttu'
# https://www.google.com/maps/search/?api=1&query=Google&query_place_id=ChIJIS56-XFmZIgRsvTzijiFReo
driver.get(url)

# Click reviews link
driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]').click()
time.sleep(5)


In [285]:
# Parse html page
response = BeautifulSoup(driver.page_source, 'html.parser')

# Find total number of reviews
# total_reviews = int(response.find('div', class_ = 'fontBodySmall').text.replace(' reviews', ''))
# total_reviews=76

# Find scroll layout
scrollable_div = driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

# Scroll as many times as necessary to load all reviews
for i in range(0, (round(total_reviews / 5 - 1))):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(1)

In [ ]:
# Re-parse html page after loading all reviews
response = BeautifulSoup(driver.page_source, 'html.parser')

print(response)

# Close the Selenium webdriver
driver.quit()

# IOPub data rate exceeded.
# The notebook server will temporarily stop sending output
# to the client in order to avoid crashing it.
# To change this limit, set the config variable
# `--NotebookApp.iopub_data_rate_limit`.

# Current values:
# NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
# NotebookApp.rate_limit_window=3.0 (secs)

In [ ]:
# Extract reviews
review_ids = [review_id['data-review-id'] for review_id in response.findAll('div', class_= 'jftiEf')]
review_dates = [review_date.text for review_date in response.findAll('span', class_ = 'rsqaWe')]
reviewers = [reviewer['aria-label'] for reviewer in response.findAll('div', class_= 'jftiEf')]
ratings = [rating['aria-label'].strip() for rating in response.findAll('span', class_ = 'kvMYJc')]
review_texts = [review_text.text for review_text in response.findAll('span', class_ = 'wiI7pd')]

In [ ]:
# Using len() function
length_using_len = len(review_ids)
print("review_ids:", length_using_len)

length_using_len = len(review_dates)
print("review_dates:", length_using_len)

length_using_len = len(reviewers)
print("reviewers:", length_using_len)

length_using_len = len(ratings)
print("ratings:", length_using_len)

length_using_len = len(review_texts)
print("review_texts:", length_using_len)

In [ ]:
# useful code to remain here for reference

# combined_list = []

# for rating, review_text in zip(ratings, review_texts):
#     combined_item = f"Rating: {rating}, Review Text: {review_text}"
#     combined_list.append(combined_item)
    
# reviews = pd.DataFrame({'Rating': [item.split(', ')[0].split(': ')[1] for item in combined_list],
#                    'Review': [item.split(', ')[1].split(': ')[1] for item in combined_list]})

# reviews

In [ ]:
combined_list = []

for rating, review_text, review_date in zip(ratings, review_texts, review_dates):
    combined_item = f"Rating: {rating}, Review Text: {review_text}, Review Date: {review_date}"
    combined_list.append(combined_item)

# Extract 'Rating,' 'Review Text,' and 'Review Date' and create a list of dictionaries
data_list = []
for item in combined_list:
    item_dict = {}
    for pair in item.split(', '):
        parts = pair.split(': ')
        key = parts[0].strip()
        value = parts[1].strip() if len(parts) > 1 else None
        if key in ['Rating', 'Review Text', 'Review Date']:
            item_dict[key] = value
    data_list.append(item_dict)

# Create a DataFrame from the list of dictionaries
reviews = pd.DataFrame(data_list)

# Display the DataFrame
reviews


In [ ]:
# Assume you already have a DataFrame named review_texts
company = 'getgo'
file_path = f'./{company}_reviews.csv'  # Use './' to save in the current directory

# Saving the DataFrame to a CSV file with the specified path
reviews.to_csv(file_path, index=False) 
